# Метод штрафов. ADMM (10 баллов)

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import itertools

__Задача 1.__ Рассмотрим задачу оптимизации SVM:

$$
\min_{x \in \mathbb{R}^d} \frac{1}{2} \|x\|_2^2 + C \sum_{i = 1}^n \max \left(1 - b_i \cdot \langle a_i, x \rangle, 0 \right).
$$

__a) (1 балл)__ Переформулируйте задачу в задачу оптимизации с ограничениями.

_Подсказка: попробуйте представить слагаемое в сумме как $s = Bx + e$. Для удобства обозначений также замените суммирование на функцию:_

$$
p(y) = \sum_{i = 1}^n \max(y_i, 0).
$$

Пусть $s_i = 1 - b_i \cdot \langle a_i, x \rangle$.
Строки $B$ выберем такие, что $B_i = -b_ia_i^T$ и возьмем единичный веткор $e = (1, \dots, 1)^T$
Тогда задача имеет вид:


$$
\min_{x \in \mathbb{R}^d, s \in \mathbb{R}^n} \quad \frac{1}{2} \|x\|_2^2 + C \cdot p(s) \\
s.t. \quad s = Bx +e
$$


__б) (1 балл)__ Для полученной задачи оптимизации с ограничениями напишите функцию Лагранжа $\mathcal{L}(x, y, \lambda)$.

$\mathcal{L}(x, y, \lambda) = \frac{1}{2} \|x\|_2^2 + C \cdot p(s) + \lambda^T(s - Bx - e)$

__в) (1 балл)__ Запишите Лагранжиан аугментированной задачи $\mathcal{L}_{\rho}(x, s, \lambda)$.




$\mathcal{L}_{\rho}(x, s, \lambda) = \frac{1}{2} \|x\|_2^2 + C \cdot p(s) + \lambda^T(s - Bx - e) + \frac{\rho}{2}\|s - Bx - e\|_2^2$



__г) (1 балл)__ Найдите $\arg \min\limits_{x \in \mathbb{R}^d} \mathcal{L}_{\rho}(x, s, \lambda)$.

$\nabla_x \left( \lambda^T (s - Bx - e) \right) = -B^T \lambda$.

$\nabla_x (\|x\|_2^2) = 2x$

$\nabla_x \left( \frac{\rho}{2} \|s - Bx - e\|_2^2 \right)  = -\rho B^T (s - Bx - e)$

$\nabla_x (\mathcal{L}_{\rho}(x, s, \lambda)) = x - B^T \lambda - \rho B^T (s - Bx - e) = 0$

$x^* = (I + \rho B^T B)^{-1} B^T (\lambda + \rho s - \rho e)$

Следовательно,
$$
\arg \min\limits_{x \in \mathbb{R}^d} \mathcal{L}_{\rho}(x, s, \lambda) = (I + \rho B^T B)^{-1} B^T (\lambda + \rho s - \rho e)
$$

__д) (3 балл)__ Найдите $\arg \min\limits_{y \in \mathbb{R}^n} \mathcal{L}_{\rho}(x, y, \lambda)$.

__е) (3 балла)__ Реализуйте метод ADMM.

**Псевдокод алгоритма**

---

_Инициализация:_

- Начальная точка $y^0 \in \mathbb{R}^n$
- Начальная точка $\lambda^0 \in \mathbb{R}^n$
- Начальный параметр штрафа $\rho \in \mathbb{R}$
- Максимальное число итераций $K$.

---

$k$_-ая итерация_:

1. Посчитайте обновление $x$:

$$
x^{k + 1} = \arg \min_{x \in \mathbb{R}^d} \mathcal{L}_{\rho} (x, y^k, \lambda^k)
$$

2. Посчитайте обновление $y$:

$$
y^{k + 1} = \arg \min_{y \in \mathbb{R}^n} \mathcal{L}_{\rho} (x^{k + 1}, y, \lambda^k)
$$

3. Посчитайте обновление $\lambda$:

$$
\lambda^{k + 1} = \lambda^k + \rho (-Bx^{k + 1} + y^{k + 1} - e)
$$


---

_Условие остановки:_
- Достигнуто максимальное число итераций $K$ или $f\left(x^k\right) < \varepsilon$

---

_Выход:_
- Полученное значение $\sum\limits_{k = 0}^K x^k$

В качестве дата-матрицы $A$ и целевого вектора $b$ рассмотрим данные из датасета [_mushrooms_](https://github.com/BRAIn-Lab-teaching/OPTIMIZATION-METHODS-COURSE/blob/ПМИ_осень_2025/Datasets/mushrooms.txt). Ниже представлена функция загрузки датасета.

In [2]:
url = "https://raw.githubusercontent.com/BRAIn-Lab-teaching/OPTIMIZATION-METHODS-COURSE/%D0%9F%D0%9C%D0%98_%D0%BE%D1%81%D0%B5%D0%BD%D1%8C_2025/Datasets/mushrooms.txt"
!wget -O mushrooms.txt "$url"

--2025-11-27 20:08:16--  https://raw.githubusercontent.com/BRAIn-Lab-teaching/OPTIMIZATION-METHODS-COURSE/%D0%9F%D0%9C%D0%98_%D0%BE%D1%81%D0%B5%D0%BD%D1%8C_2025/Datasets/mushrooms.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 879712 (859K) [text/plain]
Saving to: ‘mushrooms.txt’

mushrooms.txt       100%[===================>] 859.09K  --.-KB/s    in 0.04s   

2025-11-27 20:08:16 (19.3 MB/s) - ‘mushrooms.txt’ saved [879712/879712]



In [3]:
from sklearn.datasets import load_svmlight_file

#файл должен лежать в той же директории, что и notebook
dataset = "mushrooms.txt"

data = load_svmlight_file(dataset)
A, b = data[0].toarray(), data[1]

b = 2 * b - 3

C помощью функции ```train_test_split``` разделите датасет в отношении 4 к 1 (обучающая выборка должна быть в 4 раза больше, чем тестовая). Поставьте параметр ```random_state = 57```. В дальнейшем мы будем валидировать процесс обучения на тестовой выборке.

In [7]:
import sklearn
A_train, A_test, b_train, b_test = sklearn.model_selection.train_test_split(A, b, test_size=0.2, random_state=57)

Дополните функцию потерь.

In [ ]:
def criterion(A_train, b_train, x, C=1):
    """
    Вычисляет loss.

    Параметры:
        A_train (np.array): Матрица признаков
        b_train (np.array): Вектор меток
        x (np.array): Вектор параметров
        C (float): Коэффициент регуляризации

    Возвращает:
        loss (float): Значение функции потерь
    """

    loss = 0.5 * np.linalg.norm(x)**2 + C * np.sum(np.maximum(1 - b_train * (A_train @ x), 0))

    return loss

Допишите `ADMM`.

In [ ]:
def ADMM(A, b, criterion, x_0, y_0, lambda_0, eps=1e-8, max_iter=100, **params):
    """
    Реализация ADMM.

    Параметры:
        A (np.array): Матрица признаков
        b (np.array): Вектор меток
        criterion (Callable): Функция критерия остановки
        x_0 (np.array): Начальная точка
        y_0 (np.array): Начальная точка
        lambda_0 (np.array): Начальная точка
        eps (float): Точность сходимости (критерий остановки)
        max_iter (int): Максимальное количество итераций
        params : Именованные гиперпараметры метода
            params['C']: Параметр регуляризации
            params['rho']: Параметр штрафа

    Возвращает:
        x_k (np.array): Найденное решение
        values (list): Список значений x_k на каждой итерации
        errors (list): Список значений критерия сходимости на каждой итерации
    """

    values = []
    errors = []

    x_k = np.copy(x_0)
    y_k = np.copy(y_0)
    lmbda_k = np.copy(lambda_0)

    values.append(x_k)
    errors.append(criterion(A, b, x_k, params['C']))

    # YOUR CODE HERE

    for k in range(max_iter):

        # YOUR CODE HERE

        values.append(x_k)
        errors.append(criterion(A, b, x_k, params['C']))

        if errors[-1] < eps:
            break

    return np.mean(np.array(values)), values, errors

Запустите ADMM для $C =$ `1e-6`, `1e-4`, `1e-2`.

In [ ]:
# Ваше решение (Code)

Постройте графики сходимости ошибки.

In [ ]:
# Ваше решение (Code)

Постройте график accuracy.

In [ ]:
# Ваше решение (Code)